<a href="https://colab.research.google.com/github/pascalghanimi/Injury-Prediction-in-Runners/blob/main/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Datenextraktion für XGBoost
import pickle

with open("features.pkl", "rb") as f:
  features = pickle.load(f)

with open("features_days.pkl", "rb") as f:
  features_days = pickle.load(f)

with open("features_weeks.pkl", "rb") as f:
  features_weeks = pickle.load(f)

with open("features_objective.pkl", "rb") as f:
  features_objective = pickle.load(f)

with open("features_subjective.pkl", "rb") as f:
  features_subjective = pickle.load(f)

with open("labels.pkl", "rb") as f:
  labels = pickle.load(f)

print(features[0][0]) # greift auf ersten Athleten und erste Reihe zu
print(features_days[0][0])
print(features_weeks[0])
print(labels[0])

print(len(features[0][0])) # Anzahl der Features

[ 0.77523982  0.48379555 -0.34174298  1.12482238  4.65573275 -0.30257114
 -0.36195376  1.13221076  0.88408664  0.98946749 -1.07576228 -0.58780279
 -0.34515758 -0.21498604 -0.25818757 -0.30257114 -0.36653747 -1.10832632
 -1.12373205 -1.11938273  0.76147321 -0.58880824 -0.33963599 -0.22015974
 -0.26519341  3.1128211  -0.36131219  0.94306135  0.88408664  0.88198911
 -1.08073515 -0.58890474 -0.33382237 -0.21223522 -0.26471133 -0.29617444
 -0.36131219 -1.10795815 -1.12373205 -1.11942205  0.7667035  -0.5913292
 -0.33382237 -0.22367068 -0.26509659 -0.29617444  2.42402301  0.57403001
  0.88989186  0.9965464   0.78053094  2.39956507  5.70866175 -0.21223522
 -0.26519341  3.3763886  -0.35067995  1.13636789  0.88989186  0.88771258
  0.78584128 -0.58219763 -0.3393889  -0.21762941 -0.25818757 -0.28968273
  2.19131074  0.96442407  0.89572629  0.67074481  0.37283858 -0.47040001
 -0.01179093  0.73505066  1.36358038  0.5749299   0.54093582  1.26319022
  2.68987335 -0.05530893  0.11981333  1.16084473  1.

In [2]:
import numpy as np

X = np.vstack([features[athlete_id] for athlete_id in features])
X_days = np.vstack([features_days[athlete_id] for athlete_id in features_days])
X_weeks = np.vstack([features_weeks[athlete_id] for athlete_id in features_weeks])
X_objective = np.vstack([features_objective[athlete_id] for athlete_id in features_objective])
X_subjective = np.vstack([features_subjective[athlete_id] for athlete_id in features_subjective])

Y = np.hstack([labels[athlete_id] for athlete_id in labels])


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

X_train_days, X_temp_days, Y_train_days, Y_temp_days = train_test_split(X_days, Y, test_size=0.3, random_state=42)
X_val_days, X_test_days, Y_val_days, Y_test_days = train_test_split(X_temp_days, Y_temp_days, test_size=0.5, random_state=42)

X_train_weeks, X_temp_weeks, Y_train_weeks, Y_temp_weeks = train_test_split(X_weeks, Y, test_size=0.3, random_state=42)
X_val_weeks, X_test_weeks, Y_val_weeks, Y_test_weeks = train_test_split(X_temp_weeks, Y_temp_weeks, test_size=0.5, random_state=42)

X_train_subjective, X_temp_subjective, Y_train_subjective, Y_temp_subjective = train_test_split(X_subjective, Y, test_size=0.3, random_state=42)
X_val_subjective, X_test_subjective, Y_val_subjective, Y_test_subjective = train_test_split(X_temp_subjective, Y_temp_subjective, test_size=0.5, random_state=42)

X_train_objective, X_temp_objective, Y_train_objective, Y_temp_objective = train_test_split(X_objective, Y, test_size=0.3, random_state=42)
X_val_objective, X_test_objective, Y_val_objective, Y_test_objective = train_test_split(X_temp_objective, Y_temp_objective, test_size=0.5, random_state=42)

print(X_train.shape, X_val.shape, X_test.shape)
print(X_train_days.shape, X_val_days.shape, X_test_days.shape)
print(X_train_weeks.shape, X_val_weeks.shape, X_test_weeks.shape)
print(X_train_subjective.shape, X_val_subjective.shape, X_test_subjective.shape)
print(X_train_objective.shape, X_val_objective.shape, X_test_objective.shape)
print(Y_train.shape, Y_val.shape, Y_test.shape)


(29876, 139) (6402, 139) (6402, 139)
(29876, 70) (6402, 70) (6402, 70)
(29876, 69) (6402, 69) (6402, 69)
(29876, 48) (6402, 48) (6402, 48)
(29876, 91) (6402, 91) (6402, 91)
(29876,) (6402,) (6402,)


In [30]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
import lightgbm as lgb
import warnings

def train(X_train, Y_train, X_val, Y_val, X_test, Y_test):
  warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

  def get_training_batch(X_train, Y_train, batch_size):
    injured_indices = np.where(Y_train == 1)[0]
    uninjured_indices = np.where(Y_train == 0)[0]

    injured_sample = np.random.choice(injured_indices, size=batch_size//2, replace=True)
    uninjured_sample = np.random.choice(uninjured_indices, size=batch_size//2, replace=True)

    selected_indices = np.concatenate([injured_sample, uninjured_sample])
    np.random.shuffle(selected_indices)

    X_batch = X_train[selected_indices]
    Y_batch = Y_train[selected_indices]

    return X_batch, Y_batch

  params = {
      "objective": "binary",
      "learning_rate": 0.05,
      "max_depth": 3,
      "num_leaves": 7,
      "subsample": 0.8,
      "colsample_bytree": 0.8,
      "n_estimators": 512,
      "lambda_l2": 1.0,
      "lambda_l1": 0.5,
      "boosting_type": "gbdt",
      "verbose": -1
  }

  num_models = 9
  models = []
  batch_size = 4096

  for model_idx in range(num_models):
    X_batch, Y_batch = get_training_batch(X_train, Y_train, batch_size)
    model = lgb.LGBMClassifier(**params)
    model.fit(X_batch, Y_batch, eval_set=[(X_val, Y_val)])
    Y_pred_val = model.predict_proba(X_val)[:, 1]
    auc_roc_score = roc_auc_score(Y_val, Y_pred_val)
    Y_pred_test = model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(Y_test, Y_pred_test)

    print(f"\nLightGBM Modell {model_idx + 1} Ergebnisse:")
    print(f"Test-AUC: {test_auc:.4f}")
    models.append(model)

  calibrated_models = [
          CalibratedClassifierCV(m, method='sigmoid', cv="prefit").fit(X_val, Y_val) for m in models
  ]

  def ensemble_predict(models, X):
    probabilities = np.array([m.predict_proba(X)[:, 1] for m in models])
    return probabilities.mean(axis=0)

  Y_probab_test = ensemble_predict(calibrated_models, X_test)
  test_auc = roc_auc_score(Y_test, Y_probab_test)

  print(f"Test-AUC des LightGBM Ensembles: {test_auc:.4f}")


In [31]:
# Training mit allen Featuren
train(X_train, Y_train, X_val, Y_val, X_test, Y_test)


LightGBM Modell 1 Ergebnisse:
Test-AUC: 0.6717

LightGBM Modell 2 Ergebnisse:
Test-AUC: 0.6863

LightGBM Modell 3 Ergebnisse:
Test-AUC: 0.6927

LightGBM Modell 4 Ergebnisse:
Test-AUC: 0.6879

LightGBM Modell 5 Ergebnisse:
Test-AUC: 0.7048

LightGBM Modell 6 Ergebnisse:
Test-AUC: 0.6972

LightGBM Modell 7 Ergebnisse:
Test-AUC: 0.6886

LightGBM Modell 8 Ergebnisse:
Test-AUC: 0.6877

LightGBM Modell 9 Ergebnisse:
Test-AUC: 0.6877


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des LightGBM Ensembles: 0.7140


In [32]:
# Training mit den Tagesdaten
train(X_train_days, Y_train_days, X_val_days, Y_val_days, X_test_days, Y_test_days)


LightGBM Modell 1 Ergebnisse:
Test-AUC: 0.6747

LightGBM Modell 2 Ergebnisse:
Test-AUC: 0.6782

LightGBM Modell 3 Ergebnisse:
Test-AUC: 0.6767

LightGBM Modell 4 Ergebnisse:
Test-AUC: 0.7008

LightGBM Modell 5 Ergebnisse:
Test-AUC: 0.6524

LightGBM Modell 6 Ergebnisse:
Test-AUC: 0.6578

LightGBM Modell 7 Ergebnisse:
Test-AUC: 0.6578

LightGBM Modell 8 Ergebnisse:
Test-AUC: 0.6505

LightGBM Modell 9 Ergebnisse:
Test-AUC: 0.6827


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des LightGBM Ensembles: 0.6972


In [33]:
# Training mit den Wochendaten
train(X_train_weeks, Y_train_weeks, X_val_weeks, Y_val_weeks, X_test_weeks, Y_test_weeks)


LightGBM Modell 1 Ergebnisse:
Test-AUC: 0.6993

LightGBM Modell 2 Ergebnisse:
Test-AUC: 0.6728

LightGBM Modell 3 Ergebnisse:
Test-AUC: 0.6627

LightGBM Modell 4 Ergebnisse:
Test-AUC: 0.6802

LightGBM Modell 5 Ergebnisse:
Test-AUC: 0.6896

LightGBM Modell 6 Ergebnisse:
Test-AUC: 0.6952

LightGBM Modell 7 Ergebnisse:
Test-AUC: 0.6460

LightGBM Modell 8 Ergebnisse:
Test-AUC: 0.7032

LightGBM Modell 9 Ergebnisse:
Test-AUC: 0.6655


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des LightGBM Ensembles: 0.6984


In [34]:
# Training mit den objektiven Daten
train(X_train_objective, Y_train_objective, X_val_objective, Y_val_objective, X_test_objective, Y_test_objective)


LightGBM Modell 1 Ergebnisse:
Test-AUC: 0.6574

LightGBM Modell 2 Ergebnisse:
Test-AUC: 0.6854

LightGBM Modell 3 Ergebnisse:
Test-AUC: 0.6853

LightGBM Modell 4 Ergebnisse:
Test-AUC: 0.6908

LightGBM Modell 5 Ergebnisse:
Test-AUC: 0.6814

LightGBM Modell 6 Ergebnisse:
Test-AUC: 0.6565

LightGBM Modell 7 Ergebnisse:
Test-AUC: 0.6729

LightGBM Modell 8 Ergebnisse:
Test-AUC: 0.6802

LightGBM Modell 9 Ergebnisse:
Test-AUC: 0.6733


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des LightGBM Ensembles: 0.6984


In [35]:
# Training mit den subjektiven Daten
train(X_train_subjective, Y_train_subjective, X_val_subjective, Y_val_subjective, X_test_subjective, Y_test_subjective)


LightGBM Modell 1 Ergebnisse:
Test-AUC: 0.6116

LightGBM Modell 2 Ergebnisse:
Test-AUC: 0.6450

LightGBM Modell 3 Ergebnisse:
Test-AUC: 0.6297

LightGBM Modell 4 Ergebnisse:
Test-AUC: 0.6790

LightGBM Modell 5 Ergebnisse:
Test-AUC: 0.6536

LightGBM Modell 6 Ergebnisse:
Test-AUC: 0.6332

LightGBM Modell 7 Ergebnisse:
Test-AUC: 0.6470

LightGBM Modell 8 Ergebnisse:
Test-AUC: 0.6489

LightGBM Modell 9 Ergebnisse:
Test-AUC: 0.6219


/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.1

Test-AUC des LightGBM Ensembles: 0.6733
